In [49]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.impute import SimpleImputer 
import joblib
import pickle

In [50]:
# 1. Carregamento e limpeza dos dados
df = pd.read_csv("./termotrilho.csv", delimiter=';')
df.columns = [
    "data", "hora_utc", "precipitacao", "temperatura_ar", "umidade_ar",
    "temperatura_trilho", "radiacao_solar", "velocidade_vento"
]
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

df["precipitacao"] = df["precipitacao"].str.replace(" mm", "", regex=False).astype(float)
df["temperatura_ar"] = df["temperatura_ar"].astype(float)
df["umidade_ar"] = df["umidade_ar"].astype(float)
df["temperatura_trilho"] = df["temperatura_trilho"].astype(float)
df["radiacao_solar"] = df["radiacao_solar"].astype(float)
df["velocidade_vento"] = df["velocidade_vento"].astype(float)

# Remover registros com valor nulo em 'temperatura_trilho'
df = df.dropna(subset=["temperatura_trilho"])

# Seleção de features e target
X = df.drop(columns=["data", "hora_utc", "temperatura_trilho"])
y = df["temperatura_trilho"]

c:\AppData\Local\Temp\ipykernel_10188\2828837387.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [51]:
# 2. Seleção de features e target
X = df.drop(columns=["data", "hora_utc", "temperatura_trilho"])
y = df["temperatura_trilho"]

In [52]:
# 3. Divisão treino/teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [53]:
# 4. Definição dos modelos
modelos = {
    "Linear Regression": Pipeline([
        ("imputer", SimpleImputer(strategy="mean")), # Add imputer to pipeline
        ("scaler", StandardScaler()),
        ("regressor", LinearRegression())
    ]),
    "Random Forest": Pipeline([
        ("imputer", SimpleImputer(strategy="mean")), # Add imputer to pipeline
        ("scaler", StandardScaler()),
        ("regressor", RandomForestRegressor(random_state=42))
    ]),
    "Gradient Boosting": Pipeline([
        ("imputer", SimpleImputer(strategy="mean")), # Add imputer to pipeline
        ("scaler", StandardScaler()),
        ("regressor", GradientBoostingRegressor(random_state=42))
    ])
}

In [54]:
# 5. Treinar, avaliar e escolher o melhor modelo

resultados = {}
melhor_modelo = None
melhor_rmse = float("inf")

for nome, pipeline in modelos.items():
    pipeline.fit(X_train, y_train)
    preds = pipeline.predict(X_test)
    mae = mean_absolute_error(y_test, preds)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    resultados[nome] = {"MAE": mae, "RMSE": rmse}
    print(f"{nome} => MAE: {mae:.2f}, RMSE: {rmse:.2f}")
    
    if rmse < melhor_rmse:
        melhor_rmse = rmse
        melhor_modelo = pipeline

Linear Regression => MAE: 0.33, RMSE: 0.48
Random Forest => MAE: 0.77, RMSE: 1.57
Gradient Boosting => MAE: 0.75, RMSE: 1.51


In [55]:
# Salvar o melhor modelo com joblib
joblib.dump(melhor_modelo, "modelo_trem.pkl")
print("\nModelo salvo como 'modelo_trem.pkl'")


Modelo salvo como 'modelo_trem.pkl'


In [56]:
model = pickle.load(open("modelo_trem.pkl", "rb"))
print(f"Tipo do objeto carregado: {type(model)}")

Tipo do objeto carregado: <class 'numpy.ndarray'>
